#### Process AGCD and AWRA datasets for all the grids

In [24]:
%who

Client	 SLURMCluster	 Scheduler	 agcd_dir	 agcd_files	 allWeek_allSets	 allWeek_dict	 client	 climtas	 
cluster	 create_filepath	 da_P	 da_P_accum	 data_names	 datasets	 datetime	 day_len_dict	 delayed	 
ds_P_accum	 ds_agcd	 end_day	 end_day_dict	 field	 full_dir_path	 i	 iSet	 iWeek	 
itertools	 lat_slice	 lon_slice	 my	 nWeek	 ndays	 new_path	 np	 os	 
out_dir	 paths	 prefix	 root_path	 split_by_chunks	 start_day	 start_day_dict	 sub_dir	 sub_dir_names	 
sys	 time_chunk	 time_chunk_dict	 time_slice	 values	 week_names	 xr	 


In [4]:
31/11.6*4

10.689655172413794

In [15]:
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=8,memory="31GB")
client = Client(cluster)
cluster.scale(cores=8)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.155:38677,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers = 2, threads_per_worker = 1)
client = Client(cluster)

In [19]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.128.155:38677,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: 2 minutes ago,Total memory: 0 B


In [25]:
cluster.scale(cores = 0)

In [26]:
! squeue -u ad9701

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
              7252      main dask-wor   ad9701 CG       7:50      1 ood-n4
              7253      main dask-wor   ad9701 CG       7:50      1 ood-n5
              7227       vdi sys/dash   ad9701  R    1:30:08      1 ood-vn27


In [7]:
cluster.scale(cores = 0)

In [2]:
import sys
new_path = '/home/566/ad9701/drought_probability/'
if new_path not in sys.path:
    sys.path.append(new_path)

import xarray as xr
import numpy as np
import my_data_process_funcs as my

# creating information required for weekly accumulations
week_names = list(np.arange(2, 13, 2))
ndays = list(np.arange(14, 13*14, 14))
start_day = ['1911-01-02'] * len(week_names)

end_day = [#'2020-05-31',   #1
           '2020-05-24',   #2
           #'2020-05-31',   #3
           '2020-05-24',   #4
           '2020-05-10',   #6
           '2020-04-26',   #8
           '2020-03-29',   #10
           '2020-03-29'    #12
          ]
time_chunk = [#364,   #1 
              378,   #2
              #378,   #3
              336,   #4
              378,   #6
              336,   #8
              350,   #10
              336    #12
             ]

day_len_dict = dict(zip(week_names, ndays))
start_day_dict = dict(zip(week_names, start_day))
end_day_dict = dict(zip(week_names, end_day))
time_chunk_dict = dict(zip(week_names, time_chunk))

data_names = ['day_len', 'start_day', 'end_day', 'time_chunk']
values = [day_len_dict, start_day_dict, end_day_dict, time_chunk_dict]
allWeek_dict = dict(zip(data_names, values))

In [ ]:
out_dir = '/g/data/w97/ad9701/p_prob_analysis/weekly_data/awra/'
sub_dir = "/set1"

for i in range(2, 13, 3): #[2, 6, 8, 12]:
    temp = my.calc_agcd_accum(nWeek = i, allWeek_dict = allWeek_dict, out_dir = out_dir, sub_dir = sub_dir)
    # temp = my.calc_awra_accum(nWeek = i, allWeek_dict = allWeek_dict, file_names = file_names, var_name = var_name, out_dir = out_dir, sub_dir = sub_dir)

In [ ]:
var_name = 'qtot'
file_names = 'qtot_*.nc'
for i in range(2, 13, 3): #[2, 6, 8, 12]:
    temp = my.calc_agcd_accum(nWeek = i, allWeek_dict = allWeek_dict, out_dir = out_dir, sub_dir = sub_dir)
    # temp = my.calc_awra_accum(nWeek = i, allWeek_dict = allWeek_dict, file_names = file_names, var_name = var_name, out_dir = out_dir, sub_dir = sub_dir)

In [3]:
# Using the same workflow to generate data for alternate accumulations for the smaller SE AU data

allWeek_allSets = {}
for i in range(2, 13, 2):
    field = 'week' + str(i) + '_sets'
    allWeek_allSets.update({field: my.create_week_sets(nWeek = i, allWeek_dict = allWeek_dict)})

sub_dir_names = []
for i in range(2, 2+len(allWeek_allSets['week2_sets'])):
    #print('/set' + str(i))
    sub_dir_names.append('/set' + str(i))

out_dir = '/g/data/w97/ad9701/p_prob_analysis/temp_files/'

lat_slice = slice(-39, -32)
lon_slice = slice(139, 152)

from dask import delayed

for iWeek in [2]: #[2, 6, 8, 12]:
    for iSet in range(2, len(allWeek_allSets['week' + str(iWeek) + '_sets'])):
        set_dict = allWeek_allSets['week' + str(iWeek) + '_sets'][iSet]
        # delayed(add)(x, y)
        temp = delayed(my.calc_agcd_accum)(nWeek = iWeek, allWeek_dict = set_dict, out_dir = out_dir, sub_dir = sub_dir_names[iSet], lat_slice = lat_slice, lon_slice = lon_slice)
        # temp = my.calc_awra_accum(nWeek = i, allWeek_dict = allWeek_dict, file_names = file_names, var_name = var_name, out_dir = out_dir, sub_dir = sub_dir)

In [3]:
from dask import delayed
import climtas
import os
import datetime

allWeek_allSets = {}
for i in range(2, 13, 2):
    field = 'week' + str(i) + '_sets'
    allWeek_allSets.update({field: my.create_week_sets(nWeek = i, allWeek_dict = allWeek_dict)})

sub_dir_names = []
for i in range(2, 2+len(allWeek_allSets['week2_sets'])):
    #print('/set' + str(i))
    sub_dir_names.append('/set' + str(i))

iWeek = 2
iSet = 2
nWeek = iWeek
allWeek_dict = allWeek_allSets['week' + str(iWeek) + '_sets'][iSet]
out_dir = '/g/data/w97/ad9701/p_prob_analysis/temp_files/'
sub_dir = sub_dir_names[iSet]
agcd_dir = '/g/data/zv2/agcd/v1/precip/total/r005/01day/'
agcd_files = 'agcd_v1_precip_total_r005_daily_*.nc'
lat_slice = slice(-39, -32)
lon_slice = slice(139, 152)

ds_agcd = xr.open_mfdataset(agcd_dir + agcd_files) #, chunks = {'lat':400,'lon':400})

time_slice = slice(allWeek_dict['start_day'][nWeek], allWeek_dict['end_day'][nWeek])
time_chunk_dict = {'time':allWeek_dict['time_chunk'][nWeek]}
da_P = ds_agcd['precip'].sel(lat = lat_slice, lon = lon_slice, time = time_slice).chunk(chunks = time_chunk_dict)  # time needs to be rechunked for blocked_resample operation
da_P_accum = climtas.blocked.blocked_resample(da_P, time = 7*nWeek).sum()

full_dir_path = out_dir + 'P_week' + str(nWeek) + sub_dir
print(full_dir_path)
if not os.path.exists(full_dir_path):
    os.makedirs(full_dir_path)

# for year, data in da_P_accum.groupby('time.year'):
#     print('Writing file:' + str(year))
#     out_file = full_dir_path + '/P_week' + str(nWeek) + '_SEA_' + str(year) + '.nc'
#     delayed(climtas.io.to_netcdf_throttled)(data, f'{out_file}')#.compute()

# z.compute()

/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4


In [5]:
da_P_accum

<xarray.DataArray 'precip' (time: 2854, lat: 141, lon: 261)>
dask.array<resample_op, shape=(2854, 141, 261), dtype=float32, chunksize=(27, 141, 261), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1911-01-05T09:00:00 ... 2020-05-14T09:00:00
  * lat      (lat) float32 -39.0 -38.95 -38.9 -38.85 ... -32.1 -32.05 -32.0
  * lon      (lon) float32 139.0 139.1 139.1 139.1 ... 151.9 151.9 151.9 152.0
Attributes:
    cell_methods:                  time: sum
    units:                         mm
    analysis_version_number:       3.01
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    standard_name:                 lwe_thickness_of_precipitation_amount
    frequency:                     daily
    length_scale_for_analysis:     80.0
    grid_mapping:                  crs
    coverage_content_type:         physicalMeasurement
    valid_range:                   [     0. 100000.]

In [6]:
import itertools
def split_by_chunks(dataset):
    chunk_slices = {}
    for dim, chunks in dataset.chunks.items():
        slices = []
        start = 0
        for chunk in chunks:
            if start >= dataset.sizes[dim]:
                break
            stop = start + chunk
            slices.append(slice(start, stop))
            start = stop
        chunk_slices[dim] = slices
    for slices in itertools.product(*chunk_slices.values()):
        selection = dict(zip(chunk_slices.keys(), slices))
        yield dataset[selection]
        
def create_filepath(ds, prefix='filename', root_path="."):
    """
    Generate a filepath when given an xarray dataset
    """
    start = ds.time[0].dt.strftime("%Y-%m-%d").data
    end = ds.time[-1].dt.strftime("%Y-%m-%d").data
    # start = ds.time.data[0].strftime("%Y-%m-%d")
    # end = ds.time.data[-1].strftime("%Y-%m-%d")
    filepath = f'{root_path}/{prefix}_{start}_{end}.nc'
    return filepath

In [7]:
ds_P_accum = da_P_accum.to_dataset()
datasets = list(split_by_chunks(ds_P_accum))

In [12]:
da_P_accum

<xarray.DataArray 'precip' (time: 2854, lat: 141, lon: 261)>
dask.array<resample_op, shape=(2854, 141, 261), dtype=float32, chunksize=(27, 141, 261), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1911-01-05T09:00:00 ... 2020-05-14T09:00:00
  * lat      (lat) float32 -39.0 -38.95 -38.9 -38.85 ... -32.1 -32.05 -32.0
  * lon      (lon) float32 139.0 139.1 139.1 139.1 ... 151.9 151.9 151.9 152.0
Attributes:
    cell_methods:                  time: sum
    units:                         mm
    analysis_version_number:       3.01
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    standard_name:                 lwe_thickness_of_precipitation_amount
    frequency:                     daily
    length_scale_for_analysis:     80.0
    grid_mapping:                  crs
    coverage_content_type:         physicalMeasurement
    valid_range:                   [     0. 100000.]

In [9]:
len(datasets)

106

In [48]:
xr.testing.assert_equal(datasets[0], datasets[1])

AssertionError: Left and right Dataset objects are not equal
Differing dimensions:
    (time: 27, lat: 141, lon: 260) != (time: 27, lat: 141, lon: 1)
Differing coordinates:
L * lon      (lon) float32 139.0 139.1 139.1 139.1 ... 151.8 151.9 151.9 151.9
R * lon      (lon) float32 152.0
Differing data variables:
L   precip   (time, lat, lon) float32 dask.array<chunksize=(27, 141, 260), meta=np.ndarray>
R   precip   (time, lat, lon) float32 dask.array<chunksize=(27, 141, 1), meta=np.ndarray>

In [49]:
datasets[0]

<xarray.Dataset>
Dimensions:  (time: 27, lat: 141, lon: 260)
Coordinates:
  * time     (time) datetime64[ns] 1911-01-05T09:00:00 ... 1912-01-04T09:00:00
  * lat      (lat) float32 -39.0 -38.95 -38.9 -38.85 ... -32.1 -32.05 -32.0
  * lon      (lon) float32 139.0 139.1 139.1 139.1 ... 151.8 151.9 151.9 151.9
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(27, 141, 260), meta=np.ndarray>

In [50]:
datasets[1]

<xarray.Dataset>
Dimensions:  (time: 27, lat: 141, lon: 1)
Coordinates:
  * time     (time) datetime64[ns] 1911-01-05T09:00:00 ... 1912-01-04T09:00:00
  * lat      (lat) float32 -39.0 -38.95 -38.9 -38.85 ... -32.1 -32.05 -32.0
  * lon      (lon) float32 152.0
Data variables:
    precip   (time, lat, lon) float32 dask.array<chunksize=(27, 141, 1), meta=np.ndarray>

In [10]:
paths = [create_filepath(ds) for ds in datasets]

In [13]:
root_path = full_dir_path
prefix = 'P_week' + str(nWeek) + '_SEA_'
create_filepath(datasets[0], prefix = prefix, root_path = root_path)
# start = datasets[0].time[0].dt.strftime("%Y-%m-%d").data
# end = datasets[0].time[-1].dt.strftime("%Y-%m-%d").data
# # start = ds.time.data[0].strftime("%Y-%m-%d")
# # end = ds.time.data[-1].strftime("%Y-%m-%d")
# filepath = f'{root_path}/{prefix}_{start}_{end}.nc'
# filepath
paths = [create_filepath(ds, prefix, root_path) for ds in datasets]

In [14]:
paths

['/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1911-01-05_1912-01-04.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1912-01-18_1913-01-16.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1913-01-30_1914-01-29.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1914-02-12_1915-02-11.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1915-02-25_1916-02-24.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1916-03-09_1917-03-08.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1917-03-22_1918-03-21.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1918-04-04_1919-04-03.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1919-04-17_1920-04-15.nc',
 '/g/data/w97/ad9701/p_prob_analysis/temp_files/P_week2/set4/P_week2_SEA__1920-04-29_1921-0

In [23]:
xr.save_mfdataset(datasets=datasets, paths=paths)

In [6]:
cluster.scale(cores = 0)

In [15]:
allWeek_allSets['week2_sets'][0]

{'day_len': {2: 14},
 'time_chunk': {2: 378},
 'start_day': {2: '1911-01-03'},
 'end_day': {2: '2020-05-25'}}

In [5]:
cluster.scale(cores=0)

In [2]:
for i in range(10, 13, 2):
    temp = my.calc_agcd_accum(nWeek = i, allWeek_dict = allWeek_dict, out_dir = out_dir, sub_dir = sub_dir)

Writing file:1911


  0%|          | 0/12 [00:00<?, ?it/s]

Writing file:1912


  0%|          | 0/12 [00:00<?, ?it/s]

Writing file:1913


...

Writing file:1920


  0%|          | 0/6 [00:00<?, ?it/s]

Writing file:1921


  0%|          | 0/6 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import sys
new_path = '/home/566/ad9701/drought_probability/'
if new_path not in sys.path:
    sys.path.append(new_path)

import xarray as xr
import numpy as np
import my_data_process_funcs as my

# creating information required for weekly accumulations
week_names = list(np.arange(2, 13, 2))
ndays = list(np.arange(14, 13*14, 14))
start_day = ['1911-01-02'] * len(week_names)

end_day = [#'2020-05-31',   #1
           '2020-05-24',   #2
           #'2020-05-31',   #3
           '2020-05-24',   #4
           '2020-05-10',   #6
           '2020-04-26',   #8
           '2020-03-29',   #10
           '2020-03-29'    #12
          ]
time_chunk = [#364,   #1 
              378,   #2
              #378,   #3
              336,   #4
              378,   #6
              336,   #8
              350,   #10
              336    #12
             ]

day_len_dict = dict(zip(week_names, ndays))
start_day_dict = dict(zip(week_names, start_day))
end_day_dict = dict(zip(week_names, end_day))
time_chunk_dict = dict(zip(week_names, time_chunk))

data_names = ['day_len', 'start_day', 'end_day', 'time_chunk']
values = [day_len_dict, start_day_dict, end_day_dict, time_chunk_dict]
allWeek_dict = dict(zip(data_names, values))

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

out_dir = '/g/data/w97/ad9701/p_prob_analysis/weekly_data/awra/'
sub_dir = "/set1"

for i in range(12, 13, 2):
    temp = my.calc_agcd_accum(nWeek = i, allWeek_dict = allWeek_dict, out_dir = out_dir, sub_dir = sub_dir)

Writing file:1911


  0%|          | 0/12 [00:00<?, ?it/s]

Writing file:1912


  0%|          | 0/12 [00:00<?, ?it/s]

Writing file:1913


  0%|          | 0/12 [00:00<?, ?it/s]

Writing file:1914


  0%|          | 0/12 [00:00<?, ?it/s]

#### Scratch Space

In [3]:
import climtas
nWeek = 2
agcd_dir = '/g/data/zv2/agcd/v1/precip/total/r005/01day/'
agcd_files = 'agcd_v1_precip_total_r005_daily_*.nc'
lat_slice = slice(-44, -10)
lon_slice = slice(112, 154)

ds_agcd = xr.open_mfdataset(agcd_dir + agcd_files, chunks = {'lat':400,'lon':400})

time_slice = slice('1911-01-02', '2020-05-24') #allWeek_dict['start_day'][nWeek], allWeek_dict['end_day'][nWeek])
time_chunk_dict = {'time':allWeek_dict['time_chunk'][nWeek]}
da_P = ds_agcd['precip'].sel(lat = lat_slice, lon = lon_slice, time = time_slice).chunk(chunks = time_chunk_dict)  # time needs to be rechunked for blocked_resample operation
print(allWeek_dict['day_len'][nWeek])
#n = allWeek_dict['day_len'][nWeek]
da_P_accum = climtas.blocked.blocked_resample(da_P, time = 7*nWeek).sum()
da_P_accum

14


<xarray.DataArray 'precip' (time: 2854, lat: 681, lon: 841)>
dask.array<resample_op, shape=(2854, 681, 841), dtype=float32, chunksize=(27, 390, 400), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1911-01-02T09:00:00 ... 2020-05-11T09:00:00
  * lat      (lat) float32 -44.0 -43.95 -43.9 -43.85 ... -10.1 -10.05 -10.0
  * lon      (lon) float32 112.0 112.1 112.1 112.2 ... 153.9 153.9 153.9 154.0
Attributes:
    cell_methods:                  time: sum
    units:                         mm
    analysis_version_number:       3.01
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    standard_name:                 lwe_thickness_of_precipitation_amount
    frequency:                     daily
    length_scale_for_analysis:     80.0
    grid_mapping:                  crs
    coverage_content_type:         physicalMeasurement
    valid_range:                   [     0. 100000.]

In [4]:
for i in range(2, 13, 2):
    print(i)

2
4
6
8
10
12


In [ ]:
%who

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [1]:
x = None
x is None

True